# **토큰 인코딩 확인**

In [51]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 로드

In [52]:
train_data = pd.read_csv('/content/drive/MyDrive/GBT Hackerton/data/train_df_translate_del.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GBT Hackerton/data/test_df_translate_del.csv')

### 소분류별 많이 등장한 공통 키워드 제거

In [53]:
train_data['분류'].nunique()

56

In [54]:
# 공통 키워드를 제거하는 함수
def remove_common_keywords(df, common_keywords):
    for index, row in df.iterrows():
        keywords = row['키워드'].split(',')
        # 공통 키워드를 제외한 새로운 키워드 리스트
        filtered_keywords = [kw for kw in keywords if kw not in common_keywords]
        # 필터링된 키워드를 문자열로 다시 합침
        df.at[index, '키워드'] = ' '.join(filtered_keywords)
    return df

In [55]:
common_keywords_list = ['용인시', '경기', '서울', '경기도', '용인', '상황', '대표', '진행', '이날', '지역', '관계자', '예정', '시작', '사진', '참여',
                        '활동', '생각', '설명', '코로나19', '시장', '지원', '제공', '전국', '시민', '운영', '한국', '마련', '모습', '시민들', '준비',
                        '가능', '계획', '사업', '위치', '자리', '대상', '확인', '정도', '기록', '사람', '노력']

train_df = remove_common_keywords(train_data, common_keywords_list)
test_df = remove_common_keywords(test_data, common_keywords_list)

### 모델링

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

In [8]:
config = {
    "learning_rate": 2e-5,
    "epoch": 3,
    "batch_size": 64
}

CFG = SimpleNamespace(**config)

In [77]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('beomi/KcBERT-base')
model = BertForSequenceClassification.from_pretrained('beomi/KcBERT-base', num_labels=len(train_df['분류'].unique())).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=200):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [79]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

인코딩된 데이터셋 샘플 확인
- 2: CLS 토큰
- 3: SEP 토큰
- 0: UNK 토큰
- 1: 패딩 토큰
- 나머지 정수: 인코딩된 단어 토큰

*monologg/kobert로 인코딩한 결과*

In [69]:
train_dataset[5]

{'text': 'HL디앤아이한라 27년 선봬 아파트 브랜드 에피트 둔전역 공급 HL디앤아이한라 27년 에피트 아파트 브랜드 공급 업계 HL디앤아이한라 일원 다음달 처인구 포곡읍 금어리 에피트 둔전역 투시도 에피트 HL디앤아이한라 27년 아파트 브랜드 수도권 공급 지하 지상 최고 13개 가구 규모 가구 70% 실수요자 인기 구성 교통망 사통 팔달 둔전역 경전철 에버 라인 도보 이용 역세권 경전철 노선 이용 기흥역 지하철 수인 분당선 환승 20분 이동 용인IC 영동고속도로 진입 세종 고속도로 수도 순환 고속도 포곡 IC 인접 국지도 포곡 도로 금어 IC 개통 수도 전역 이동 편리 주변 본점 로마트 포곡농협 명주병원 둔전체육공원 삼계공원 경안천 수변공원 포곡체육공원 정수산 교육시설 도보 통학 둔전초교 포곡중 영문중 고림중 용인고 포곡고 고림고교 인접 단지 처인구 중심지 글로벌 반도체 육성 정부 국가산업단지 첨단 반도체 국가 산업 단지 반도체 클러스터 일반 산업 단지 조성 연결 철도 고속도로 광역 교통망 확충 추진 업계 클러스터 수도 반도체 중심지 수요 신규 주택 에피트 둔전역 산단 첨단 반도체 국가 인근 아파트 만큼 직주 근접 인기 예상',
 'input_ids': tensor([   2,    0,    0,    0, 6809, 6434,    0,    0, 5455,    0,    0,    0,
         6809, 6434, 5455, 6874,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, 6809, 6434,    0, 5455,    0,    0, 7459,    0,
         5332, 5533, 5332,  207,   11,    0,    0,    0,    0,    0,    0,    0,
            0,    0, 6014,    0,    0,    0,    0,    0,    0,    0,    0,

In [74]:
list_input_ids = [2,    0,    0,    0, 6809, 6434,    0,    0, 5455,    0,    0,    0,
         6809, 6434, 5455, 6874,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, 6809, 6434,    0, 5455,    0,    0, 7459,    0,
         5332, 5533, 5332,  207,   11,    0,    0,    0,    0,    0,    0,    0,
            0,    0, 6014,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0, 5442,    0,    0,    0,
            0,  299,    0,    0,    0, 5860,    0,  299,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 5790,
            0,    0, 5548, 6287,    0, 7230,    0,    0, 6287, 5504, 6517, 5790,
         6287,    0, 7129, 6517, 5790,    0,    0,    0, 5442,    0,    0,    0,
         7462, 6874,    0,    0, 6287,    0,    0,    0, 7286,    0,    0,    0,
            0, 6287, 5504,    0, 6809, 6157,    0,    0,    0,    0,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]

for idx in list_input_ids:
        print('{} ----> {}'.format(idx, tokenizer.decode([idx])))

2 ----> [CLS]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
6809 ----> 아파트
6434 ----> 브랜드
0 ----> [UNK]
0 ----> [UNK]
5455 ----> 공급
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
6809 ----> 아파트
6434 ----> 브랜드
5455 ----> 공급
6874 ----> 업계
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
6809 ----> 아파트
6434 ----> 브랜드
0 ----> [UNK]
5455 ----> 공급
0 ----> [UNK]
0 ----> [UNK]
7459 ----> 최고
0 ----> [UNK]
5332 ----> 가구
5533 ----> 규모
5332 ----> 가구
207 ----> 70
11 ----> %
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
6014 ----> 라인
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
0 ----> [UNK]
5442 ----> 고속도로
0 ----> [UNK]
0 ----> [UNK]
0 

In [76]:
text = 'HL디앤아이한라 27년 선봬 아파트 브랜드 에피트 둔전역 공급 HL디앤아이한라 27년 에피트 아파트 브랜드 공급 업계 HL디앤아이한라 일원 다음달 처인구 포곡읍 금어리 에피트 둔전역 투시도 에피트 HL디앤아이한라 27년 아파트 브랜드 수도권 공급 지하 지상 최고 13개 가구 규모 가구 70% 실수요자 인기 구성 교통망 사통 팔달 둔전역 경전철 에버 라인 도보 이용 역세권 경전철 노선 이용 기흥역 지하철 수인 분당선 환승 20분 이동 용인IC 영동고속도로 진입 세종 고속도로 수도 순환 고속도 포곡 IC 인접 국지도 포곡 도로 금어 IC 개통 수도 전역 이동 편리 주변 본점 로마트 포곡농협 명주병원 둔전체육공원 삼계공원 경안천 수변공원 포곡체육공원 정수산 교육시설 도보 통학 둔전초교 포곡중 영문중 고림중 용인고 포곡고 고림고교 인접 단지 처인구 중심지 글로벌 반도체 육성 정부 국가산업단지 첨단 반도체 국가 산업 단지 반도체 클러스터 일반 산업 단지 조성 연결 철도 고속도로 광역 교통망 확충 추진 업계 클러스터 수도 반도체 중심지 수요 신규 주택 에피트 둔전역 산단 첨단 반도체 국가 인근 아파트 만큼 직주 근접 인기 예상'

# 인코딩
encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=200,
    padding='max_length',
    truncation=True,
    return_tensors='pt',
)

# 인코딩된 결과 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# 디코딩
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("Decoded Text:", decoded_text)

# 원본 텍스트와 비교
print("Original Text:", text)

Decoded Text: 아파트 브랜드 공급 아파트 브랜드 공급 업계 아파트 브랜드 공급 최고 가구 규모 가구 70 % 라인 고속도로 IC 도로 IC 단지 글로벌 반도체 정부 반도체 국가 산업 단지 반도체 일반 산업 단지 고속도로 추진 업계 반도체 주택 반도체 국가 아파트 만큼
Original Text: HL디앤아이한라 27년 선봬 아파트 브랜드 에피트 둔전역 공급 HL디앤아이한라 27년 에피트 아파트 브랜드 공급 업계 HL디앤아이한라 일원 다음달 처인구 포곡읍 금어리 에피트 둔전역 투시도 에피트 HL디앤아이한라 27년 아파트 브랜드 수도권 공급 지하 지상 최고 13개 가구 규모 가구 70% 실수요자 인기 구성 교통망 사통 팔달 둔전역 경전철 에버 라인 도보 이용 역세권 경전철 노선 이용 기흥역 지하철 수인 분당선 환승 20분 이동 용인IC 영동고속도로 진입 세종 고속도로 수도 순환 고속도 포곡 IC 인접 국지도 포곡 도로 금어 IC 개통 수도 전역 이동 편리 주변 본점 로마트 포곡농협 명주병원 둔전체육공원 삼계공원 경안천 수변공원 포곡체육공원 정수산 교육시설 도보 통학 둔전초교 포곡중 영문중 고림중 용인고 포곡고 고림고교 인접 단지 처인구 중심지 글로벌 반도체 육성 정부 국가산업단지 첨단 반도체 국가 산업 단지 반도체 클러스터 일반 산업 단지 조성 연결 철도 고속도로 광역 교통망 확충 추진 업계 클러스터 수도 반도체 중심지 수요 신규 주택 에피트 둔전역 산단 첨단 반도체 국가 인근 아파트 만큼 직주 근접 인기 예상


*beomi/KcBERT-base로 인코딩한 결과*

In [80]:
train_dataset[5]

{'text': '8억 부동산 거래 거짓 신고 적발 부과 부동산 거래 거짓 신고 의심 사례 지분거래 기획부동산 편법 지분 거래 2618건 특별조사 결과 406명 거짓 신고자 적발 6000만 과태료 부과 20일 공인 중개사법 위반 행위 37명 허위 거래 관련자 수사 의뢰 2월 7월 다운계약신고 계약 거짓 특수관계 매매 불가 거래 대금 사례 거짓 신고 의심 금전거래 신고 허위신고 자금 조달 계획서 증여 의심 거래 집중적 조사 자체 기획부동산 상시 모니터링 활용 토지거래허가구역 조사 결과 토지 거래 허가 구역 실제 매매 계약 체결 허가 회피 근저당 설정 중개행위 행위 중개 행위 중개 보수 초과 수수 공인 중개사법 위반 행위 37명 수사기관 수사 요청 구체적 시세 조작 상향 주택 담보 대출 한도 실제 거래 금액 업계약 체결 감면 양도 소득세 실제 거래 가격 가격 계약서 이중계약 다운계약 체결 지연 신고 계약일자 기타 거짓 신고 거짓 조장방조 자료미제출 위법 행위 406명 6천만 과태료 부과 적발 사례 A씨 토지거래허가구역 지정 가평군 임야 기획부동산 혐의 의심 주식회사 실제 소유 거래계약 체결 허가 회피 목적 근저당 설정 토지 거래 허가 구역 해제 소유권 사실 적발 수사기관 수사 요청 C씨 주택 파주시 소재 D씨 실거래 신고 조사 결과 실제 거래 금액 2억 가량 신고 사실 적발 매도자 매수자 과태료 1200만 부과 매수자 아파트 소재 매수 실거래 신고 매수자 아버지 매매대금 대납 사실 혐의 증여세 탈루 관할 세무 관서 통보 혐의점 거래 서류 혐의 부동산 매도 매수자 가족 친척 특수관계 주변 시세 가격 거래 신고 451건 세무관서 통보 세무조사 요청 유형별 기타 특수 매매 거래 가격 의심 거래 대금 불가 대물 변제 편법증여 의심 161건',
 'input_ids': tensor([    2, 27467,  8807, 11815,  8192,  9710,  2524,  4235, 21653,  8807,
         11815,  8192,  9710,  9107, 

In [82]:
list_input_ids = [    2, 27467,  8807, 11815,  8192,  9710,  2524,  4235, 21653,  8807,
         11815,  8192,  9710,  9107, 13390, 27558, 13181, 13894, 26362, 16025,
         27558, 11815, 15801,  9168,  4164, 11317,  8553,  8713,  8914,  4539,
          4268,  8192,  9710,  4105,  2524,  4235, 22625,  4263,  4049, 29551,
          4018, 21653,  8255,  4046, 17918,  2635,  4064, 27890, 11584, 13450,
         27551,  4268, 11010, 11815, 29175,  8201, 25433, 18163, 25268, 17341,
         20215, 13524, 12446,  8192, 12652,  9352, 18600, 13724, 11815,   809,
          4301, 13390,  8192,  9710,  9107,   400,  4203, 13181,  9710, 11010,
         13524, 14471,  2572,  4182, 11011,  4072, 17968,  9107, 11815, 11997,
          4022,  8254, 10736, 13894, 26362,  1801,  4039, 28137,  4025,  5190,
         16267, 22390, 13181, 21413, 18327,  8254,  8713, 22390, 11815, 14224,
         11581,  9597, 18600, 12446, 28646, 14224, 20103,   393,  4488,  4081,
          1849,  4036,  2635,  4064, 12175, 13450,  2635,  4064, 13450,  2635,
          4064,  8197, 27327, 14758, 17918,  2635,  4064, 27890, 11584, 13450,
         27551,  4268,  8201, 10052,  8201, 12585, 16396,  4022, 21400,  8265,
          1801,  4573, 13546,   797,  4010, 10137,  3354,  4029,  9597, 11815,
         13781,  2245, 20215, 28646,   208,  4063, 18166, 29028,  9597, 11815,
          9286,  9286, 12446,  4072,  9720, 20215, 17341, 20215, 28646, 29256,
          9710, 12446,  4046,  4105, 15705,  8192,  9710,  8192, 11486,  4015,
          4194, 11717,  4090,  4231,  4208, 16594, 13450,  8914,  4539,     3]

for idx in list_input_ids:
        print('{} ----> {}'.format(idx, tokenizer.decode([idx])))

2 ----> [CLS]
27467 ----> 8억
8807 ----> 부동산
11815 ----> 거래
8192 ----> 거짓
9710 ----> 신고
2524 ----> 적
4235 ----> ##발
21653 ----> 부과
8807 ----> 부동산
11815 ----> 거래
8192 ----> 거짓
9710 ----> 신고
9107 ----> 의심
13390 ----> 사례
27558 ----> 지분
13181 ----> ##거래
13894 ----> 기획
26362 ----> ##부동산
16025 ----> 편법
27558 ----> 지분
11815 ----> 거래
15801 ----> 26
9168 ----> ##18
4164 ----> ##건
11317 ----> 특별
8553 ----> ##조사
8713 ----> 결과
8914 ----> 40
4539 ----> ##6
4268 ----> ##명
8192 ----> 거짓
9710 ----> 신고
4105 ----> ##자
2524 ----> 적
4235 ----> ##발
22625 ----> 600
4263 ----> ##0
4049 ----> ##만
29551 ----> 과태
4018 ----> ##료
21653 ----> 부과
8255 ----> 20
4046 ----> ##일
17918 ----> 공인
2635 ----> 중
4064 ----> ##개
27890 ----> ##사법
11584 ----> 위반
13450 ----> 행위
27551 ----> 37
4268 ----> ##명
11010 ----> 허위
11815 ----> 거래
29175 ----> 관련자
8201 ----> 수사
25433 ----> 의뢰
18163 ----> 2월
25268 ----> 7월
17341 ----> 다운
20215 ----> ##계약
13524 ----> ##신고
12446 ----> 계약
8192 ----> 거짓
12652 ----> 특수
9352 ----> ##관계
18600 ----> 매

In [83]:
text = 'HL디앤아이한라 27년 선봬 아파트 브랜드 에피트 둔전역 공급 HL디앤아이한라 27년 에피트 아파트 브랜드 공급 업계 HL디앤아이한라 일원 다음달 처인구 포곡읍 금어리 에피트 둔전역 투시도 에피트 HL디앤아이한라 27년 아파트 브랜드 수도권 공급 지하 지상 최고 13개 가구 규모 가구 70% 실수요자 인기 구성 교통망 사통 팔달 둔전역 경전철 에버 라인 도보 이용 역세권 경전철 노선 이용 기흥역 지하철 수인 분당선 환승 20분 이동 용인IC 영동고속도로 진입 세종 고속도로 수도 순환 고속도 포곡 IC 인접 국지도 포곡 도로 금어 IC 개통 수도 전역 이동 편리 주변 본점 로마트 포곡농협 명주병원 둔전체육공원 삼계공원 경안천 수변공원 포곡체육공원 정수산 교육시설 도보 통학 둔전초교 포곡중 영문중 고림중 용인고 포곡고 고림고교 인접 단지 처인구 중심지 글로벌 반도체 육성 정부 국가산업단지 첨단 반도체 국가 산업 단지 반도체 클러스터 일반 산업 단지 조성 연결 철도 고속도로 광역 교통망 확충 추진 업계 클러스터 수도 반도체 중심지 수요 신규 주택 에피트 둔전역 산단 첨단 반도체 국가 인근 아파트 만큼 직주 근접 인기 예상'

# 인코딩
encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=200,
    padding='max_length',
    truncation=True,
    return_tensors='pt',
)

# 인코딩된 결과 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# 디코딩
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("Decoded Text:", decoded_text)

# 원본 텍스트와 비교
print("Original Text:", text)

Decoded Text: HL디앤아이한라 27년 선봬 아파트 브랜드 에피트 둔전역 공급 HL디앤아이한라 27년 에피트 아파트 브랜드 공급 업계 HL디앤아이한라 일원 다음달 처인구 포곡읍 금어리 에피트 둔전역 투시도 에피트 HL디앤아이한라 27년 아파트 브랜드 수도권 공급 지하 지상 최고 13개 가구 규모 가구 70 % 실수요자 인기 구성 교통망 사통 팔달 둔전역 경전철 에버 라인 도보 이용 역세권 경전철 노선 이용 기흥역 지하철 수인 분당선 환승 20분 이동 용인IC 영동고속도로 진입 세종 고속도로 수도 순환 고속도 포곡 IC 인접 국지도 포곡 도로 금어 IC 개통 수도 전역 이동 편리 주변 본점 로마트 포곡농협 명주병원 둔전체육공원 삼계공원 경안천 수변공원
Original Text: HL디앤아이한라 27년 선봬 아파트 브랜드 에피트 둔전역 공급 HL디앤아이한라 27년 에피트 아파트 브랜드 공급 업계 HL디앤아이한라 일원 다음달 처인구 포곡읍 금어리 에피트 둔전역 투시도 에피트 HL디앤아이한라 27년 아파트 브랜드 수도권 공급 지하 지상 최고 13개 가구 규모 가구 70% 실수요자 인기 구성 교통망 사통 팔달 둔전역 경전철 에버 라인 도보 이용 역세권 경전철 노선 이용 기흥역 지하철 수인 분당선 환승 20분 이동 용인IC 영동고속도로 진입 세종 고속도로 수도 순환 고속도 포곡 IC 인접 국지도 포곡 도로 금어 IC 개통 수도 전역 이동 편리 주변 본점 로마트 포곡농협 명주병원 둔전체육공원 삼계공원 경안천 수변공원 포곡체육공원 정수산 교육시설 도보 통학 둔전초교 포곡중 영문중 고림중 용인고 포곡고 고림고교 인접 단지 처인구 중심지 글로벌 반도체 육성 정부 국가산업단지 첨단 반도체 국가 산업 단지 반도체 클러스터 일반 산업 단지 조성 연결 철도 고속도로 광역 교통망 확충 추진 업계 클러스터 수도 반도체 중심지 수요 신규 주택 에피트 둔전역 산단 첨단 반도체 국가 인근 아파트 만큼 직주 근접 인기 예상


단어가 토크나이저의 어휘 사전에 있는지 확인
- ex) KoBert의 경우 '거래량'이 어휘 사전에 없었는데 KcBert는 있는 것을 확인

In [48]:
# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('beomi/KcBERT-base')

# 확인할 단어
word = '거래량'  # 예시 단어

# 단어를 토크나이징
token_ids = tokenizer.encode(word, add_special_tokens=False)

# 결과 확인
print(f"'{word}'의 토큰 ID: {token_ids}")

# 어휘에 포함되어 있는지 확인
if token_ids[0] == tokenizer.unk_token_id:
    print(f"'{word}'는 어휘에 없습니다.")
else:
    print(f"'{word}'는 어휘에 있습니다.")

'거래량'의 토큰 ID: [11815, 4667]
'거래량'는 어휘에 있습니다.


In [ ]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

In [ ]:
# 학습
model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

In [ ]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [03:46<00:00,  1.61it/s]


In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/GBT Hackerton/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/GBT Hackerton/data/submission_KoBert_1003.csv", encoding='UTF-8-sig', index=False)

In [ ]:
result = pd.read_csv("/content/drive/MyDrive/GBT Hackerton/data/submission_KoBert_1003.csv")
result.head()

,ID,분류
0,TEST_00000,지역
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:국회_정당
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역


In [ ]:
result['분류'].value_counts()

,count
분류,
지역,12086
경제:부동산,1362
사회:사건_사고,1038
경제:반도체,947
사회:교육_시험,502
정치:국회_정당,453
경제:산업_기업,451
경제:취업_창업,439
사회:사회일반,369
